<a href="https://colab.research.google.com/github/Jeffrey2659/LatinTeamTest/blob/main/LatinDataSetGenerator_(Not_Working).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install git+https://github.com/sjgallagher2/PyWORDS
!pip install unidecode
import random
import requests
import csv
import os
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import re
import unicodedata
#import pywords.lookup as lookup
#from urllib.request import urlopen
#pip install html5lib
#pip install lxml

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
PATH= 'drive/Shareddrives/AI in Classics/GreekABASAModel/DataSetGenerator'


os.chdir(PATH)# sets the current active directory to the current folder
!pwd
# IF YOU ENCOUNTER AN ERROR FIRST TRY RESTARTING RUNTIME

Mounted at /content/drive
/content/drive/Shareddrives/AI in Classics/GreekABASAModel/DataSetGenerator


In [ ]:
#Creating the functions
import pandas as pd

#load in the dataframes for functions
lemmaDF = pd.read_csv("Latin_Lemmatizer/lemmas.csv", sep = "{")
del lemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Latin_Lemmatizer/parses.csv", sep = "{")
del parsesDF["Unnamed: 0"]
#outputs list of matching IDs or an empty list
def get_id(word):
    #search the lemmaDF for the word
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if(row["id"] not in output):
            output.append(row["id"])
    return(output)
def get_id_parse(word, input):
    #search parseDF for instances and add to output if it isn't already in there
    output = input
    df = parsesDF.loc[parsesDF["bare_text"] == word]
    for index, row in df.iterrows():
        id = row["id"]
        if id not in output:
            output.append(id)
    return output

#outouts dictionary of ids to the valid words
def get_words_help(ids):
    output = {}
    if(type(ids) == list):
        for id in ids:
            curOut = []
            df = parsesDF.loc[parsesDF["id"] == id]
            for index, row in df.iterrows():
                #can add a checker for POS or add info about POS
                curOut.append(row["bare_text"])
            output[id] = curOut
        return output
       #for each id in the list, grab all the words with matching ID from parses and put in "outputs"
       #outputs is dict of ID --> parses with ID
    else:
        print("Something went wrong...")
        return {}
    #enter FPP
def get_id_full(word):
    ids = get_id(word)
    ids = get_id_parse(word, ids)
    return ids
def get_words(word):
    ids = get_id_full(word)
    return get_words_help(ids)
def check_enclitic(row, text):
    print("out of date")
def get_def(id):
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for index, row in df.iterrows():
        output = (row["definition"])
    return output
def get_lang(id):
    #returns ltn or grk
    output = ""
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        output = row["lang_id"]
    if(output == 3 ):
        output = "ltn"
    else:
        output = "grk"
    return output
def get_all_ids():
    #returns a list of all noun ids
    output = []
    #for every id in lemmas, add it to output
    for index, row in lemmaDF.iterrows():
        out = row["id"]
        #if an int, add it to the output
        if(type(out) == type(1)):
            output.append(out)
    return output

def make_dict_csv(ids, outputFile = "outputs/nounsDictionary.csv", allowNulls = True):
    #ids is the list of ids for all words - check for nouns in here
    FPP = []
    POS = []
    DICTENTRY = []
    SOURCE = []
    for id in ids:
        pos = ""
        fpp = ""
        dictEntry = ""
        source = "morpheus"
        text = {}
        df = parsesDF.loc[parsesDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        #if no words in parses, we can't make an entry for it -- current outlook
        if(df.empty):
            continue
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6]
                #make the nom, singular --> text[0]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]
                #hold the plural genative just in case
                if(morph[2] == "s" and (morph[7] == "g")):
                    if(text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                        text[6] = row["bare_text"]
            #adj case
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if(type(row["misc_features"]) == type(1.0) ):
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "m")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "f")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "n")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"]
            #backup 4PP for verbs stored in text[6] - for weird words like futurus
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"]
            if((row["morph_code"][0] == "v")):
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(type(row["misc_features"]) == type(1.0)):
                    if(morph) == "v1spia---":
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
                    if(morph) == "v--pna---":
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]
                    if(morph) == "v1sria---":
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
                    if(morph) == "v3spia---":
                        if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                            text[4] = row["bare_text"]
                    if(morph) == "v1spip---":
                        if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                            text[5] = row["bare_text"]
             #all other cases - simplest, just has 1 form
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
        #set FPP to text[0]
        df2 = lemmaDF.loc[lemmaDF["id"] == id]
        if(pos == "noun"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            if(text[1] == "NULL"):
                text[1] = text[6]
            dictEntry = text[0] + ", " + text[1] + ", " + text[2]
            fpp = text[0]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        elif(pos == "verb"):
            #change which type by checking if we have a normal FPP
            if(text[0]!= "NULL"):
                kind = "regular"
            elif(text[5] != "NULL"):
                kind = "deponent"
            else:
                kind = "impersonal"
                #nested switch block! Yay!
            #fix edge case - 4PP is weird
            if(text[3] == "NULL"):
                    text[3] = text[6]
            match(kind):
                case "regular":
                    fpp = text[0]
                    dictEntry = text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                case "deponent":
                    fpp = text[5]
                    dictEntry = text[5] + ", " + text[1] + ", " + text[3]
                case "impersonal":
                    fpp = text[4]
                    dictEntry = text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            if(fpp == "NULL"):
                for index, row in df2.iterrows():
                    fpp = row["bare_text"]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)

        elif(pos == "adj"):
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + ", " + text[1] + ", " + text[2]
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)
        else:
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
            fpp = text[0]
            dictEntry = text[0] + " indecl."
            if("NULL" not in dictEntry or allowNulls):
                FPP.append(fpp)
                POS.append(pos)
                DICTENTRY.append(dictEntry)

    #end of function stuff
    out_map = {
        "FPP": FPP,
        "POS": POS,
        "Dictionary Entry": DICTENTRY,
        "Source": SOURCE
    }
    output = pd.DataFrame(out_map)
    output.to_csv(outputFile, sep = "\t")






#function from word -> dictionary entry
def get_dict(word, show_def = False):
    ids = get_id_full(word)
        #list of dictionary entries
    output = {}
    for id in ids:
        lang = get_lang(id)
        if(lang == "grk"):
            continue
        pos = ""
        dictEntry = ""
        rowText = ""
        morph = ""
        text = {}
        definition = get_def(id)
        df = parsesDF.loc[parsesDF["id"] == id]
        for i in range(10):
                if(not i in text):
                    text[i] = "NULL"
        for index, row in df.iterrows():
            #preset text to be all null
            rowText = row["bare_text"]
            morph = row["morph_code"]
            #noun case
                #check required to prevent substantive adj from outputting as nouns
            if (row["morph_code"][0] == "n" and (pos == "" or pos == "noun")):
                pos = "noun"
                text[2] = row["morph_code"][6]
                #make the nom, singular --> text[0]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "n")):
                    #have to hard-code around que enclitic
                    if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):

                        text[0] = row["bare_text"]
                #make the gen, singular --> text[1]
                if((row["morph_code"][2] == "s") and (row["morph_code"][7] == "g")):
                    if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                        text[1] = row["bare_text"]

            #adj case
            if (row["morph_code"][0] == "a"):
                pos = "adj"
                #for each, need the singular, positive, nominative - without enclitics
                if(type(row["misc_features"]) == type(1.0) ):
                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "m")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "f")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]

                    if((row["morph_code"][2] == "s")
                    and (row["morph_code"][7] == "n")
                    and (row["morph_code"][6] == "n")
                    and (row["morph_code"][8] == "p" or row["morph_code"][8] == "-")):
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
            #verb case - need t morph code for the 4PP
            if(row["morph_code"] == "t-sr-pmn-") and (text[3] == "NULL" or len(row["bare_text"])<len(text[3])):
                text[3] = row["bare_text"]
            #backup 4th PP for verbs stored in text[6]
            if(row["morph_code"] == "t-sf-amn-") and (text[6] == "NULL" or len(row["bare_text"])<len(text[6])):
                text[6] = row["bare_text"]
            if((row["morph_code"][0] == "v")):
                pos = "verb"
                kind = "regular"
                morph = row["morph_code"]
                curWord = row["bare_text"]
                #make 3 cases: regular, deponent, and inpersonal - may be more ask Celano
                #0: present first person singular active indicative: v1spia---
                #1: present active infinitive: v--pna---
                #2: perfect first person singular active indicative: v1sria---
                #3: t-sr-pmn- is the morph code --> has 3PP if deponent
                #4: present third person singular active indicative - for impersonal verbs
                #5: grab stuff with deponent voice tag v1spip---
                if(type(row["misc_features"]) == type(1.0)):
                    if(morph) == "v1spia---":
                        if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
                    if(morph) == "v--pna---":
                        if(text[1] == "NULL" or len(row["bare_text"])<len(text[1])):
                            text[1] = row["bare_text"]
                    if(morph) == "v1sria---":
                        if(text[2] == "NULL" or len(row["bare_text"])<len(text[2])):
                            text[2] = row["bare_text"]
                    if(morph) == "v3spia---":
                        if(text[4] == "NULL" or len(row["bare_text"])<len(text[4])):
                            text[4] = row["bare_text"]
                    if(morph) == "v1spip---":
                        if(text[5] == "NULL" or len(row["bare_text"])<len(text[5])):
                            text[5] = row["bare_text"]
            #all other cases
            if(morph[0] == "d"):
                pos = "adv"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                            text[0] = row["bare_text"]
            if(morph[0] == "c"):
                pos = "conj"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                     text[0] = row["bare_text"]
            if(morph[0] == "r"):
                pos = "adpostion"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "p"):
                pos = "pronoun"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "m"):
                pos = "numeral"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "i"):
                pos = "interjection"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "e" ):
                pos = "exclaimation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
            if(morph[0] == "u"):
                pos = "punctuation"
                if(text[0] == "NULL" or len(row["bare_text"])<len(text[0])):
                    text[0] = row["bare_text"]
        #preventing errors <3
        if(type(definition) != type("str") or len(definition) == 0):
            definition = "No Definiton"
            #case for when FPP isn't found in parses
        if(text[0] == "NULL"):
            df2 = lemmaDF.loc[lemmaDF["id"] == id]
            for index, row in df2.iterrows():
                text[0] = row["bare_text"]
        #make dictionary entry string
        match(pos):
            #make sure each of those exists before trying to output
            case "noun":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ". - " + (definition)
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2]
            case "adj":
                if(show_def):
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + " - " + definition
                else:
                    dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2]
            case "verb":
                #change which type by checking if we have a normal FPP
                if(text[0]!= "NULL"):
                    kind = "regular"
                elif(text[5] != "NULL"):
                    kind = "deponent"
                else:
                    kind = "impersonal"
                    #nested switch block! Yay!
                #solve the edge case!
                if(text[3] == "NULL"):
                    text[3] = text[6]
                match(kind):
                    case "regular":
                        if(show_def):
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[0] + ", " + text[1] + ", "+ text[2] + ", " + text[3]
                    case "deponent":
                        if(show_def):
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[5] + ", " + text[1] + ", " + text[3]
                    case "impersonal":
                        if(show_def):
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2] + ", " + text[3] + "- " + definition
                        else:
                            dictEntry = pos + ": " + text[4] + ", " + text[1] +", " + text[2]+  ", "+ text[3]
            case other:
                dictEntry = pos + ": " + text[0]
                if(show_def):
                    dictEntry += " " + definition
        output[id] = (dictEntry)
    return output

In [ ]:
#new text block for getForm
#load in the dataframes for functions
lemmaDF = pd.read_csv("Latin_Lemmatizer/lemmas.csv", sep = "{")
del lemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Latin_Lemmatizer//parses.csv", sep = "{")
del parsesDF["Unnamed: 0"]
def get_POS(id):
    #returns the part of speech of the word
    output = []
    df = parsesDF.loc[parsesDF["id"] == id]
    for(index, row) in df.iterrows():
        out = row["morph_code"][0]
        if out not in output:
            output.append(out)
    return output
def get_form(word = "NULL", id = "NULL", case = "NULL",
              number = "NULL", gender = "NULL", mood = "NULL",
              person = "NULL", tense = "NULL", voice = "NULL",
                degree = "NULL", alt_dialects = False, wanted_pos = "NULL"):
    #check if word is null - use ID in that case
    if(word == "NULL" and id == "NULL"):
        print("Please enter a word or an ID")
        return "ERROR: Please enter a word or an ID"
    if(word!= "NULL" and id !="NULL"):
            print("Please enter a word or an ID, not both")
            return "ERROR: Please enter a word or an ID, not both"
    if(word != "NULL"):
        ids = get_id_full(word)
        if(len(ids) == 0):
            print("No matching words found")
            return "ERROR: No matching words"
    if(id != "NULL"):
        ids = [id]
    for id in ids:
        #if you find a valid output in one id, you're good. If not, we continue on
        output = ""
        posTag = "---------"
        pos = get_POS(id)
        #if the word is a noun
        if("n" in pos):
            #noun
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v"

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"

            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"

            #word assigned to output if it exists
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            # number
            if (number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            # gender
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
        if("v" in pos):
            #verb
                #mood
            if(mood == "indicative"):
                mood = "i"
            if(mood == "subjunctive"):
                mood = "s"
            if(mood == "imperative"):
                mood = "m"
            if(mood == "infinitive"):
                mood = "n"
            if(mood == "participle"):
                mood = "p"
            if(mood == "gerund"):
                mood = "d"
            if(mood == "gerundive"):
                mood = "g"

                #person
            if(person == "first"):
                person = "1"
            if(person == "second"):
                person = "2"
            if(person == "third"):
                person = "3"

                #tense
            if(tense == "present"):
                tense = "p"
            if(tense == "imperfect"):
                tense = "i"
            if(tense == "perfect"):
                tense = "r"
            if(tense == "pluperfect"):
                tense = "l"
            if(tense == "future perfect"):
                tense = "t"
            if(tense == "future"):
                tense = "f"

                #voice
            if(voice == "active"):
                voice = "a"
            if(voice == "passive"):
                voice = "p"
            if(voice == "deponent"):
                voice = "d"

                #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
                # mood
            if(mood != "NULL"):
                posTag = posTag[:4] + mood + posTag[5:]
            # person
            if(person != "NULL"):
                posTag = posTag[:1] + person + posTag[2:]
            # tense
            if(tense != "NULL"):
                posTag = posTag[:3] + tense + posTag[4:]
            # voice
            if(voice != "NULL"):
                posTag = posTag[:5] + voice + posTag[6:]
            # number
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
        if("a" in pos):
                #adj case
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v"
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
            #degree
            if(degree == "positive"):
                degree = "p"
            if(degree == "comparative"):
                degree = "c"
            if(degree == "superlative"):
                degree = "s"
            if(degree != "NULL"):
                posTag = posTag[:8] + gender
        #set the pos to the wanted pos if it exists
        #change full words to their specific letters
        if(wanted_pos == "noun"):
            wanted_pos = "n"
        if(wanted_pos == "verb"):
            wanted_pos = "v"
        if(wanted_pos == "adjective"):
            wanted_pos = "a"
        if(wanted_pos == "adverb"):
            wanted_pos = "d"
        if(wanted_pos == "conjunction"):
            wanted_pos = "c"
        if(wanted_pos == "pronoun"):
            wanted_pos = "p"
        if(wanted_pos == "numeral"):
            wanted_pos = "m"
        if(wanted_pos == "interjection"):
            wanted_pos = "i"
        if(wanted_pos == "exclaimation"):
            wanted_pos = "e"
        if(wanted_pos == "punctuation"):
            wanted_pos = "u"
        #if the wanted pos is not null, change the posTag to match
        if(wanted_pos != "NULL"):
            posTag = wanted_pos + posTag[1:]
        #now actually find that word in parses
        df = parsesDF.loc[parsesDF["id"] == id]
        for index, row in df.iterrows():
            fits = True
            if(wanted_pos == "v"):
                if(row["morph_code"][0] == "v" or row["morph_code"][0] == "t"):
                    fits = True
                else:
                    fits = False
                for i in range(1, 8):
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
            else:
                for i in range(8):
                    #check if alt dialects allowed
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results
            if fits:
                output = row["bare_text"]

        #before repeating for other ids, return the output if it exists
        if(output != ""):
            return output

    return "ERROR: Nothing found with those parameters"

In [ ]:
def get_FPP(word = "NULL", id = "NULL", pos = "NULL"):
    #use char pos not full word
    if (word != "NULL"):
        ids = get_id_full(word)
        parts_of_speech = []
    if (id != "NULL"):
        ids = [id]

        #remove duplicates
    output = {}
    for id in ids:
        output[id] = ""
        if (pos != "NULL"):
            parts_of_speech = [pos]
        else:
            parts_of_speech = get_POS(id)
        if("n" in parts_of_speech):
            temp = get_form(id = id, case = "nominative", number = "singular")
            if("ERROR" in temp):
                temp = get_form(id = id, case = "nominative", number = "plural")
            if("ERROR" not in temp):
                output[id] = temp
        if("v" in parts_of_speech or "t" in parts_of_speech):
            temp = get_form(id = id, mood = "indicative", person = "first", number = "singular", tense = "present", voice = "active")
            if("ERROR" in temp):
                #no singular
                temp = get_form(id = id, mood = "indicative", person = "first", number = "plural", tense = "present", voice = "active")
            if("ERROR" in temp):
                #impersonal
                temp = get_form(id = id, mood = "indicative", person = "third", number = "singular", tense = "present", voice = "active")
            if("ERROR" in temp):
                #impersonal and no sing
                temp = get_form(id = id, mood = "indicative", person = "third", number = "plural", tense = "present", voice = "active")
            if("ERROR" in temp):
                #deponent
                temp = get_form(id = id, mood = "indicative", person = "first", number = "singular", tense = "present")
            if("ERROR" not in temp):
                output[id] = temp
        if("a" in parts_of_speech):
            temp = get_form(id = id, case = "nominative", number = "singular", degree = "positive", gender = "masculine")
            if("ERROR" in temp):
                temp = get_form(id = id, case = "nominative", number = "singular", gender = "masculine")
            if("ERROR" not in temp):
                output[id] = temp

    return output

In [ ]:

import pandas as pd
import string
from unidecode import unidecode
#DECLARE VARIABLES
grkLemmaDF = pd.read_csv("Latin_Lemmatizer/lemmas.csv", sep = "{")
del grkLemmaDF['Unnamed: 0']
latLemmaDF = pd.read_csv("Latin_Lemmatizer/lemmas.csv", sep = "{")
del latLemmaDF['Unnamed: 0']
parsesDF = pd.read_csv("Latin_Lemmatizer//parses.csv", sep = "{")
del parsesDF['Unnamed: 0']

#FUNCTIONS

def clean_input(word):
    # Remove accents
    word = unidecode(word)
    # Remove punctuation
    word = word.translate(str.maketrans('', '', string.punctuation))
    # Remove spaces
    word = word.strip()
    # Convert to lowercase
    word = word.lower()
    return word
def get_id_greek(word):
    lemmaDF = grkLemmaDF
    #search the lemmaDF for the word
    word = word.lower()
    output = []
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if(row["id"] not in output):
            output.append(row["id"])
    return(output)

def get_POS_greek(id):
    #returns the part of speech of the word
    lemmaDF = grkLemmaDF
    output = []
    df = lemmaDF.loc[lemmaDF["id"] == id]
    for(index, row) in df.iterrows():
        out = row["morph_code"][0]
        if out not in output:
            output.append(out)
    return output
def get_greek_form(word = "NULL", id = "NULL", case = "NULL",
              number = "NULL", gender = "NULL", mood = "NULL",
              person = "NULL", tense = "NULL", voice = "NULL",
                degree = "NULL", wanted_pos = "NULL"):
    #check if word is null - use ID in that case
    lemmaDF = grkLemmaDF
    if(word == "NULL" and id == "NULL"):
        print("Please enter a word or an ID")
        return "ERROR: Please enter a word or an ID"
    if(word!= "NULL" and id !="NULL"):
            print("Please enter a word or an ID, not both")
            return "ERROR: Please enter a word or an ID, not both"
    if(word != "NULL"):
        ids = get_id_greek(word)
        if(len(ids) == 0):
            print("No matching words found")
            return "ERROR: No matching words"
    if(id != "NULL"):
        ids = [id]
    for id in ids:
        #if you find a valid output in one id, you're good. If not, we continue on
        output = ""
        posTag = "---------"
        pos = get_POS_greek(id)
        #if the word is a noun
        if("n" in pos):
            #noun
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v"

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"

            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"

            #word assigned to output if it exists
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            # number
            if (number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            # gender
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
        if("v" in pos):
            #verb
                #mood
            if(mood == "indicative"):
                mood = "i"
            if(mood == "subjunctive"):
                mood = "s"
            if(mood == "imperative"):
                mood = "m"
            if(mood == "infinitive"):
                mood = "n"
            if(mood == "participle"):
                mood = "p"
            if(mood == "gerund"):
                mood = "d"
            if(mood == "gerundive"):
                mood = "g"

                #person
            if(person == "first"):
                person = "1"
            if(person == "second"):
                person = "2"
            if(person == "third"):
                person = "3"

                #tense
            if(tense == "present"):
                tense = "p"
            if(tense == "imperfect"):
                tense = "i"
            if(tense == "perfect"):
                tense = "r"
            if(tense == "pluperfect"):
                tense = "l"
            if(tense == "future perfect"):
                tense = "t"
            if(tense == "future"):
                tense = "f"

                #voice
            if(voice == "active"):
                voice = "a"
            if(voice == "passive"):
                voice = "p"
            if(voice == "deponent"):
                voice = "d"

                #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
                # mood
            if(mood != "NULL"):
                posTag = posTag[:4] + mood + posTag[5:]
            # person
            if(person != "NULL"):
                posTag = posTag[:1] + person + posTag[2:]
            # tense
            if(tense != "NULL"):
                posTag = posTag[:3] + tense + posTag[4:]
            # voice
            if(voice != "NULL"):
                posTag = posTag[:5] + voice + posTag[6:]
            # number
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
        if("a" in pos):
                #adj case

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
            #degree
            if(degree == "positive"):
                degree = "p"
            if(degree == "comparative"):
                degree = "c"
            if(degree == "superlative"):
                degree = "s"
            if(degree != "NULL"):
                posTag = posTag[:8] + gender
        #set the pos to the wanted pos if it exists
        #change full words to their specific letters
        if(wanted_pos == "noun"):
            wanted_pos = "n"
        if(wanted_pos == "verb"):
            wanted_pos = "v"
        if(wanted_pos == "adjective"):
            wanted_pos = "a"
        if(wanted_pos == "adverb"):
            wanted_pos = "d"
        if(wanted_pos == "conjunction"):
            wanted_pos = "c"
        if(wanted_pos == "pronoun"):
            wanted_pos = "p"
        if(wanted_pos == "numeral"):
            wanted_pos = "m"
        if(wanted_pos == "interjection"):
            wanted_pos = "i"
        if(wanted_pos == "exclaimation"):
            wanted_pos = "e"
        if(wanted_pos == "punctuation"):
            wanted_pos = "u"
        #if the wanted pos is not null, change the posTag to match
        if(wanted_pos != "NULL"):
            posTag = wanted_pos + posTag[1:]
        #now actually find that word in parses
        df = lemmaDF.loc[lemmaDF["id"] == id]
        for index, row in df.iterrows():
            fits = True
            if(wanted_pos == "v"):
                if(row["morph_code"][0] == "v" or row["morph_code"][0] == "t"):
                    fits = True
                else:
                    fits = False
                for i in range(1, 8):
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results

            else:
                for i in range(8):
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                        #this is the case that we disclude it from results
            if fits:
                output = row["bare_text"]

        #before repeating for other ids, return the output if it exists
        if(output != ""):
            return output

    return "ERROR: Nothing found with those parameters"


def get_id_parse_latin(word, input):
    #search parseDF for instances and add to output if it isn't already in there
    output = input
    df = parsesDF.loc[parsesDF["bare_text"] == word]
    for index, row in df.iterrows():
        id = row["id"]
        if id not in output:
            output.append(id)
    return output
def get_id_latin(word):
    #search the lemmaDF for the word
    output = []
    lemmaDF = latLemmaDF
    #this is the dataframe of all rows with the word
    df = lemmaDF.loc[lemmaDF["bare_text"] == word]
        #iterrows works, but the others dont. ok whatever
    for index, row in df.iterrows():
        if(row["id"] not in output):
            output.append(row["id"])
    return(output)
def get_id_full_latin(word):
    ids = get_id_latin(word)
    ids = get_id_parse_latin(word, ids)
    return ids
def get_POS_latin(id):
     #returns the part of speech of the word
    output = []
    df = parsesDF.loc[parsesDF["id"] == id]
    for(index, row) in df.iterrows():
        out = row["morph_code"][0]
        if out not in output:
            output.append(out)
    return output
def get_latin_form(word = "NULL", id = "NULL", case = "NULL",
              number = "NULL", gender = "NULL", mood = "NULL",
              person = "NULL", tense = "NULL", voice = "NULL",
                degree = "NULL", alt_dialects = False, wanted_pos = "NULL"):
    #check if word is null - use ID in that case
    if(word == "NULL" and id == "NULL"):
        print("Please enter a word or an ID")
        return "ERROR: Please enter a word or an ID"
    if(word!= "NULL" and id !="NULL"):
            print("Please enter a word or an ID, not both")
            return "ERROR: Please enter a word or an ID, not both"
    if(word != "NULL"):
        word = clean_input(word)
        ids = get_id_full_latin(word)
        if(len(ids) == 0):
            print("No matching words found")
            return "ERROR: No matching words"
    if(id != "NULL"):
        ids = [id]
    for id in ids:
        #if you find a valid output in one id, you're good. If not, we continue on
        output = ""
        posTag = "---------"
        pos = get_POS_latin(id)
        #if the word is a noun
        if("n" in pos):
            #noun
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v"

            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"

            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"

            #word assigned to output if it exists
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            # number
            if (number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            # gender
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
        if("v" in pos):
            #verb
                #mood
            if(mood == "indicative"):
                mood = "i"
            if(mood == "subjunctive"):
                mood = "s"
            if(mood == "imperative"):
                mood = "m"
            if(mood == "infinitive"):
                mood = "n"
            if(mood == "participle"):
                mood = "p"
            if(mood == "gerund"):
                mood = "d"
            if(mood == "gerundive"):
                mood = "g"

                #person
            if(person == "first"):
                person = "1"
            if(person == "second"):
                person = "2"
            if(person == "third"):
                person = "3"

                #tense
            if(tense == "present"):
                tense = "p"
            if(tense == "imperfect"):
                tense = "i"
            if(tense == "perfect"):
                tense = "r"
            if(tense == "pluperfect"):
                tense = "l"
            if(tense == "future perfect"):
                tense = "t"
            if(tense == "future"):
                tense = "f"

                #voice
            if(voice == "active"):
                voice = "a"
            if(voice == "passive"):
                voice = "p"
            if(voice == "deponent"):
                voice = "d"

                #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
                # mood
            if(mood != "NULL"):
                posTag = posTag[:4] + mood + posTag[5:]
            # person
            if(person != "NULL"):
                posTag = posTag[:1] + person + posTag[2:]
            # tense
            if(tense != "NULL"):
                posTag = posTag[:3] + tense + posTag[4:]
            # voice
            if(voice != "NULL"):
                posTag = posTag[:5] + voice + posTag[6:]
            # number
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
        if("a" in pos):
                #adj case
            #case
            if (case == "nominative"):
                case = "n"
            if (case == "genative"):
                case = "g"
            if (case == "dative"):
                case = "d"
            if (case == "accusative"):
                case = "a"
            if (case == "ablative"):
                case = "b"
            if (case == "locative"):
                case = "l"
            if (case == "vocative"):
                case = "v"
            if (case != "NULL"):
                # include case in search
                posTag = posTag[:7] + case + posTag[8:]
            #number
            if (number == "singular"):
                number = "s"
            if (number == "plural"):
                number = "p"
            if(number != "NULL"):
                posTag = posTag[:2] + number + posTag[3:]
            #gender
            if(gender == "masculine"):
                gender = "m"
            if (gender == "feminine"):
                gender = "f"
            if (gender == "neuter"):
                gender = "n"
            if(gender != "NULL"):
                posTag = posTag[:6] + gender + posTag[7:]
            #degree
            if(degree == "positive"):
                degree = "p"
            if(degree == "comparative"):
                degree = "c"
            if(degree == "superlative"):
                degree = "s"
            if(degree != "NULL"):
                posTag = posTag[:8] + gender
        #set the pos to the wanted pos if it exists
        #change full words to their specific letters
        if(wanted_pos == "noun"):
            wanted_pos = "n"
        if(wanted_pos == "verb"):
            wanted_pos = "v"
        if(wanted_pos == "adjective"):
            wanted_pos = "a"
        if(wanted_pos == "adverb"):
            wanted_pos = "d"
        if(wanted_pos == "conjunction"):
            wanted_pos = "c"
        if(wanted_pos == "pronoun"):
            wanted_pos = "p"
        if(wanted_pos == "numeral"):
            wanted_pos = "m"
        if(wanted_pos == "interjection"):
            wanted_pos = "i"
        if(wanted_pos == "exclaimation"):
            wanted_pos = "e"
        if(wanted_pos == "punctuation"):
            wanted_pos = "u"
        #if the wanted pos is not null, change the posTag to match
        if(wanted_pos != "NULL"):
            posTag = wanted_pos + posTag[1:]
        #now actually find that word in parses
        df = parsesDF.loc[parsesDF["id"] == id]
        for index, row in df.iterrows():
            fits = True
            if(wanted_pos == "v"):
                if(row["morph_code"][0] == "v" or row["morph_code"][0] == "t"):
                    fits = True
                else:
                    fits = False
                for i in range(1, 8):
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
            else:
                for i in range(8):
                    #check if alt dialects allowed
                    if(not alt_dialects):
                        if(type(row["misc_features"]) != type(1.0) or type(row["dialects"]) != type(1.0)):
                            fits = False
                    char = posTag[i]
                    if(char != "-"):
                        #morph code is wrong or output is longer than old output or it has misc_features or alt dialect
                        if(row["morph_code"][i] != char or (output != "" and len(output)<len(row["bare_text"]))):
                            fits = False
                            break
                        #this is the case that we disclude it from results
            if fits:
                output = row["bare_text"]

        #before repeating for other ids, return the output if it exists
        if(output != ""):
            #change to i
            output = output.replace("j", "i")
            return output
    if(not alt_dialects):
        output = get_latin_form(word=word, id="NULL", case=case, voice=voice,
                                number=number, gender=gender, degree=degree,
                                wanted_pos=wanted_pos, mood=mood, person=person,
                                tense=tense, alt_dialects=True)
    else:
        output = "ERROR: Nothing found with those parameters"
    return output


if(__name__ == '__main__'):
    #os.chdir("Individual/Leipzig-Research/Lemmatizer-GRK")

    print("hello")
    # print(get_greek_form("λεγω", mood = "indicative", tense = "present", voice = "active", person = "third", number = "singular"))
    # print(get_latin_form("mater", gender = "f", number = "p", case = "a"))
    # print(get_latin_form("senex", gender = "m", number = "s", case = "d"))
    # print(get_latin_form("pulchra", wanted_pos= "a", case = "a", number = "s", degree= "p", gender = "f"))
    print(get_latin_form("curro", mood = "indicative", tense = "perfect", voice = "active", person = "first", number = "p"))

hello
adcurrimus


In [ ]:
#check if the code runs in another block
# print(get_id("mater"))
# print(get_words_help([46163]))
#print(get_dict("dictus", True))
#print(get_latin_form(word = "mater", case = "b", number = "s"))
#print(get_latin_form(word = "fluit", mood = "i", tense = "p", voice = "a", number = "s"))
#print(get_latin_form(word = "celer", gender = "f", number = "s", case = "a", degree = "p"))
    #fixed romanus giving romanum instead of orum - needed to disallow dialect differences
#print(get_latin_form(word = "Romanus", case = "g", number = "p", wanted_pos= "noun"))
#print(get_latin_form(word = "Romanus", case = "b", number = "s", wanted_pos= "a", gender = "f"))


print(get_POS_latin(list(get_FPP("bonus"))[0])[0])
print(get_latin_form(word = "bonus", case = "nominative", number = "s", gender = "masculine"))
print(get_latin_form(word = "viri", case = "genative", number = "s"))
print(get_latin_form(word = "corpus", case = "d", number = "s"))
print(get_latin_form(word = "corpus", case = "a", number = "s"))
print(get_latin_form(word = "corpus", case = "b", number = "s"))
print(get_latin_form(word = "corpus", case = "v", number = "s"))
print(get_latin_form(word = "corpus", case = "n", number = "p"))
print(get_latin_form(word = "corpus", case = "g", number = "p"))
print(get_latin_form(word = "corpus", case = "d", number = "p"))
print(get_latin_form(word = "corpus", case = "a", number = "p"))
print(get_latin_form(word = "corpus", case = "b", number = "p"))
print(get_latin_form(word = "corpus", case = "v", number = "p"))
get_FPP("servus")


a
bonus
viri
corpori
corpus
corpore
corpus
corpora
corporum
corporibus
corpora
corporibus
corpora


{51470: 'seruus', 51471: 'seruus'}

In [ ]:
import numpy as np
def latin_dict(word):
  sing_pl =  ['singular' , 'plural']

  part_of_speech = get_POS_latin(list(get_FPP(word))[0])[0]
  print("Part of Speech: ", part_of_speech)

  data = []
  if part_of_speech == "n":
    case_list = ['nominative', 'genative', 'dative', 'accusative', 'ablative', 'vocative']
    for noun_case in case_list:
      for number in sing_pl:
        data.append(get_latin_form(word, case = noun_case, number = number))

    return pd.DataFrame(data=np.array(data).reshape(6, 2), index=case_list,
                    columns={sing_pl[0]: 'Singular', sing_pl[1]: 'Plural'}).rename_axis('Case')


  elif part_of_speech == "v" or part_of_speech == "t":
    person_list = ['first', 'second', 'third']
    for person in person_list:
      for number in sing_pl:
        data.append(get_latin_form(word, person = person, number = number, voice = 'active', mood='indicative',tense = "present"))

    return pd.DataFrame(data=np.array(data).reshape(3, 2), index=person_list,
                            columns={sing_pl[0]: 'Singular', sing_pl[1]: 'Plural'}).rename_axis("Person")


  elif part_of_speech == "a":
    data = {'word': []}
    case_list = ['nominative', 'genative', 'dative', 'accusative', 'ablative', 'vocative']
    gender_list = ['masculine', 'feminine', 'neuter']
    multi_index = pd.MultiIndex.from_product([case_list, sing_pl, gender_list], names=['case', 'number', 'gender'])
    for noun_case in case_list:
      for number in sing_pl:
        for gender in gender_list:
          data['word'].append(get_latin_form(word, case = noun_case, number = number, gender = gender))
    df = pd.DataFrame(data, index=multi_index)
    return df

  #elif part_of_speech == "d":

latin_dict('bonus')

Part of Speech:  a


word
case       number   gender            
nominative singular masculine    bonus
                    feminine      bona
                    neuter       bonum
           plural   masculine     boni
                    feminine     bonae
                    neuter        bona
genative   singular masculine     boni
                    feminine     bonae
                    neuter        boni
           plural   masculine  bonorum
                    feminine   bonarum
                    neuter     bonorum
dative     singular masculine     bono
                    feminine     bonae
                    neuter        bono
           plural   masculine    bonis
                    feminine     bonis
                    neuter       bonis
accusative singular masculine    bonum
                    feminine     bonam
                    neuter       bonum
           plural   masculine    bonos
                    feminine     bonas
                    neuter        bona
ablative   singular masculine     bono
                    feminine      bona
                    neuter        bono
           plural   masculine    bonis
                    feminine     bonis
                    neuter       bonis
vocative   singular masculine     bone
                    feminine      bona
                    neuter        bone
           plural   masculine     boni
                    feminine     bonae
                    neuter        bona

In [ ]:
latin_dict('video') #CHANGING letter V into U
                    #2nd Conjugation

Part of Speech:  v


,singular,plural
Person,,
first,videone,uidemus
second,uiden,videtin
third,uidet,uident


In [ ]:
latin_dict('capio') #3rd Conjugation

Part of Speech:  v


,singular,plural
Person,,
first,capio,capimus
second,capis,capitis
third,capit,capiunt


In [ ]:
latin_dict('audio') #4th Conjugation

Part of Speech:  v


,singular,plural
Person,,
first,audio,audimus
second,audis,auditin
third,audit,audiunt


In [ ]:
display(latin_dict('diligo') )#Third conjugation

display(latin_dict('laudo')) #First conjugation

display(latin_dict('lego')) #Third conjugation

display(latin_dict('pugno')) #First conjugation

display(latin_dict('dimitto')) #Third conjugation

display(latin_dict('portendo')) #Third conjugation)

display(latin_dict('opinor')) #DEPONENT VERB

 # latin_dict('vulnero') #COULD NOT BE FOUND #First conjugation


Part of Speech:  t


,singular,plural
Person,,
first,diligo,diligimus
second,diligis,diligitis
third,diligit,diligunt


Part of Speech:  t


,singular,plural
Person,,
first,laudo,laudamus
second,laudas,laudatis
third,laudat,laudant


Part of Speech:  t


...

,singular,plural
Person,,
first,dimitto,dimittimus
second,dimittis,dimittitis
third,dimittit,dimittunt


Part of Speech:  v


,singular,plural
Person,,
first,portendo,ERROR: Nothing found with those parameters
second,ERROR: Nothing found with those parameters,ERROR: Nothing found with those parameters
third,portendit,portendunt


Part of Speech:  v


,singular,plural
Person,,
first,ERROR: Nothing found with those parameters,ERROR: Nothing found with those parameters
second,ERROR: Nothing found with those parameters,ERROR: Nothing found with those parameters
third,ERROR: Nothing found with those parameters,ERROR: Nothing found with those parameters


In [ ]:
display(latin_dict('fructus') )#4th declension
display(latin_dict('respublica')) #5th declension
display(latin_dict('vulnus')) #3rd declension

IndexError: list index out of range

In [ ]:
latin_verbs = [
    'amo', 'porto', 'voco', 'laudo', 'dono',
    'moneo', 'habeo', 'video', 'debeo', 'doceo',
    'duco', 'capio', 'facio', 'lego', 'vinco',
    'audio', 'sentio', 'dormio', 'venio', 'cupio',
    'canto', 'celebro', 'rogo', 'respondeo', 'credo',
    'scribo', 'aperio', 'clamo', 'curro', 'vendo',
    'mitto', 'paro', 'peto', 'vinco', 'teneo',
    'venio', 'aperio', 'colo', 'mitto', 'facio',
    'pugno', 'capio', 'contendo', 'dico', 'edo',
    'accipio', 'spero', 'fugio', 'gaudeo', 'creo',
    'impedio', 'arcesso', 'aperio', 'nuntio', 'portendo',
    'spiro', 'facio', 'consulo', 'traho',
    'vinco', 'dimitto', 'opinor', 'libero', 'timeo',
    'doleo', 'expugno', 'capesso', 'disco', 'mitto',
    'nego', 'dico', 'relinquo', 'facio', 'contendo',
    'venio', 'excito', 'amitto', 'capio', 'transeo',
    'reperio', 'invado', 'facio', 'praesideo', 'tango',
    'sentio', 'appono', 'incipio', 'arbitror', 'frango',
    'capio', 'efficio', 'doceo', 'oppono', 'pugno'
]

count = 0
for i in latin_verbs:
  print(count)
  display(latin_dict(i))
  count += 1


In [ ]:
latin_adjectives = ["bonus",
"gratus", "fortis", "celer", "malus",
"rudis", "crudelis"]

count = 0
for i in latin_adjectives:
  print(count)
  display(latin_dict(i))
  count += 1


In [ ]:
!pip install cltk
from IPython.lib.pretty import Breakable

import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

#test2 #allot easier proably just going to use this
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()


def generate_variations(word,language='greek',show=False):
      # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
      #DECLINES WORDS
      #test words
      #'Ῥωμαῖος' romans
      #'μισώ' hate
      #'εἰμί' are to be
      #word = word.lower()

# takes in a word as a string and uses the collatinus decliner to return
# the different forms of the word. returns words which is a list that contains
# tuples formated like so: [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
  def decline(word):
    decliner = CollatinusDecliner()
    words = decliner.decline(word)
    return words

  # takes in a declined word object (list of words in the format returned by the decliner function) and
  # formats it so that each word is with the correct type (nominative, genative, etc.)
  # currently only works with nouns that have one gender. needs to be expanded so it can make
  # dictionaries for multiple genders and word types.
  def fill_dicts(word, neutral = False, masc = False, fem = False):
      singular_dict = {}
      plural_dict = {}
      noun_cases = ["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"]
      for k in noun_cases:
        singular_dict[k] = None
        plural_dict[k] = None

      for i in range(0, len(word)):

        #check if verb
        if word[i][1][0] == "v":
          #check if singular verb
          if word[i][1][1:] == "1spia---":
            singular_dict["First Person"] = (word[i][0])
          if word[i][1][1:] == "2spia---":
            singular_dict["Second Person"] = (word[i][0])
          if word[i][1][1:] == "3spia---":
            singular_dict["Third Person"] = (word[i][0])
          #check if plural verb
          if word[i][1][1:] == "1ppia---":
            plural_dict["First Person"] = (word[i][0])
          if word[i][1][1:] == "2ppia---":
            plural_dict["Second Person"] = (word[i][0])
          if word[i][1][1:] == "3ppia---":
            plural_dict["Third Person"] = (word[i][0])

        # check if singular
        elif word[i][1][0] == "-":
          if word[i][1][2] == "s":
            if word[i][1][7] == "n":
              if singular_dict["Nominative"] == None:
                singular_dict["Nominative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "g":
              if singular_dict["Genative"] == None:
                singular_dict["Genative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "d":
              if singular_dict["Dative"] == None:
                singular_dict["Dative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "a":
              if singular_dict["Accusative"] == None:
                singular_dict["Accusative"] = (word[i][0])
            elif word[i][1][7] == "b":
              if singular_dict["Ablative"] == None:
                singular_dict["Ablative"] = (word[i][0])
            elif word[i][1][7] == "v":
              if singular_dict["Vocative"] == None:
                singular_dict["Vocative"] = (word[i][0])
          # check if plural
          elif word[i][1][2] == "p":
            if word[i][1][7] == "n":
              if plural_dict["Nominative"] == None:
                plural_dict["Nominative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "g":
              if plural_dict["Genative"] == None:
                plural_dict["Genative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "d":
              if plural_dict["Dative"] == None:
                plural_dict["Dative"] = (word[i][0])
              else: continue
            elif word[i][1][7] == "a":
              if plural_dict["Accusative"] == None:
                plural_dict["Accusative"] = (word[i][0])
            elif word[i][1][7] == "b":
              if plural_dict["Ablative"] == None:
                plural_dict["Ablative"] = (word[i][0])
            elif word[i][1][7] == "v":
              if plural_dict["Vocative"] == None:
                plural_dict["Vocative"] = (word[i][0])

      #fix for bugged nouns by appending endings to nominative case
      if len(singular_dict) == 6 and len(singular_dict["Genative"]) < 3 and len(singular_dict["Dative"]) < 3:
        for key in singular_dict.keys():
          if key != "Nominative" and key != "Vocative":
            singular_dict[key] = singular_dict["Nominative"] + singular_dict[key]
        for key in singular_dict.keys():
            plural_dict[key] = singular_dict["Nominative"] + plural_dict[key]

      #FIX for bugged verbs by getting rid of None Values
      #if len(singular_dict) > 6


      return singular_dict, plural_dict



  def make_df(word_tup, noun = False):
    print(word_tup)

    #gender = set()
    #for i in range(0,len(word)):
    #  gender.add(word[i][1][6])
    # runs if the word has no gendered attributes
    #if '-' in gender or 'n' in gender:
    singular_dict, plural_dict = fill_dicts(word_tup, True)
    print(singular_dict, plural_dict)
    print()

    if word_tup[1][1][0] == "v":
        dict_verb = {"Person": ["First Person", "Second Person", "Third Person"], "Singular":[], "Plural":[]}
        dict_verb["Singular"].append(singular_dict["First Person"])
        dict_verb["Singular"].append(singular_dict["Second Person"])
        dict_verb["Singular"].append(singular_dict["Third Person"])
        dict_verb["Plural"].append(plural_dict["First Person"])
        dict_verb["Plural"].append(plural_dict["Second Person"])
        dict_verb["Plural"].append(plural_dict["Third Person"])
        df = pd.DataFrame(dict_verb)
        df = df.set_index("Person")
        return df, "verb"

    if word_tup[1][1][0] == "-":
      dict_overall = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
      dict_overall["Singular"].append(singular_dict["Nominative"])
      dict_overall["Singular"].append(singular_dict["Genative"])
      dict_overall["Singular"].append(singular_dict["Dative"])
      dict_overall["Singular"].append(singular_dict["Accusative"])
      dict_overall["Singular"].append(singular_dict["Ablative"])
      dict_overall["Singular"].append(singular_dict["Vocative"])

      dict_overall["Plural"].append(plural_dict["Nominative"])
      dict_overall["Plural"].append(plural_dict["Genative"])
      dict_overall["Plural"].append(plural_dict["Dative"])
      dict_overall["Plural"].append(plural_dict["Accusative"])
      dict_overall["Plural"].append(plural_dict["Ablative"])
      dict_overall["Plural"].append(plural_dict["Vocative"])
      df = pd.DataFrame(dict_overall)
      df = df.set_index('Case')
      return df, "noun"
    if 'm' in gender:
          dict_masc = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}

    else:
      print("not valid at the moment")
      print(gender)
      return -1


  df, part_of_speech = make_df(decline(word))



#print(make_df(word))


# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy

      ##display(df)
      ##CREATE A FOLDER
  root = 'latin_variations_new'
  os.makedirs(f'{root}/{part_of_speech}/{word}/',exist_ok=True)
  # NAN ARE REPALCED BY SPACE CHARACTER
  df.to_csv(f'{root}/{part_of_speech}/{word}/{word}.csv')

  print(part_of_speech)
  if(show):display(df)

  ## READ
  #print('READ FROM FILE ------------------------------------')
  #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
#generate_variations('Ῥωμαῖος','greek',show=True)
generate_variations('liber','latin',show=True)

#test words
#'Ῥωμαῖος' romans
#'μισώ' hate
#'εἰμί' is/are to be
#ελληνικά greeks
#odium hatred
#romanus
#odio hate
#απεχθάνομαι
#lacedaemon

# TO SHOW THE DATAFRAME PASS SHOW AS TRUE





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.8/849.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 46.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 

INFO:CLTK:Cloning 'lat_models_cltk' from 'https://github.com/cltk/lat_models_cltk.git'


['lat_text_perseus', 'lat_treebank_perseus', 'lat_text_latin_library', 'phi5', 'phi7', 'latin_proper_names_cltk', 'lat_models_cltk', 'latin_pos_lemmata_cltk', 'latin_treebank_index_thomisticus', 'latin_lexica_perseus', 'latin_training_set_sentence_cltk', 'latin_word2vec_cltk', 'latin_text_antique_digiliblt', 'latin_text_corpus_grammaticorum_latinorum', 'latin_text_poeti_ditalia', 'lat_text_tesserae', 'cltk_lat_lewis_elementary_lexicon']
[('liber', '--s---mn-'), ('liber', '--s---mv-'), ('liberum', '--s---ma-'), ('liberi', '--s---mg-'), ('libero', '--s---md-'), ('libero', '--s---mb-'), ('liberi', '--p---mn-'), ('liberi', '--p---mv-'), ('liberos', '--p---ma-'), ('liberorum', '--p---mg-'), ('liberis', '--p---md-'), ('liberis', '--p---mb-'), ('libera', '--s---fn-'), ('libera', '--s---fv-'), ('liberam', '--s---fa-'), ('liberae', '--s---fg-'), ('liberae', '--s---fd-'), ('libera', '--s---fb-'), ('liberae', '--p---fn-'), ('liberae', '--p---fv-'), ('liberas', '--p---fa-'), ('liberarum', '--p---f

,Singular,Plural
Case,,
Nominative,liber,liberi
Genative,liberi,liberorum
Dative,libero,liberis
Accusative,liberum,liberos
Ablative,libero,liberis
Vocative,liber,liberi


In [26]:
from IPython.lib.pretty import Breakable
import pandas as pd

#test2 #allot easier proably just going to use this
#creates inflections for nouns and adjectives
#creates conjugations for verbs
def strip_accents_and_lowercase(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').lower()


def generate_variations(word,language='greek',show=False):
      # CREATES A FOLDER STRUCTURE THAT CONTAINS WORDS ORGANIZED BY PART OF SPEECH
      #DECLINES WORDS
      #test words
      #'Ῥωμαῖος' romans
      #'μισώ' hate
      #'εἰμί' are to be
      #word = word.lower()

  def make_df(word, noun = False):

    def latin_dict(word):
      sing_pl =  ['singular' , 'plural']

      part_of_speech = get_POS_latin(list(get_FPP(word))[0])[0]
      print(part_of_speech)

      data = []
      if part_of_speech == "n":
        case_list = ['nominative', 'genative', 'dative', 'accusative', 'ablative', 'vocative']
        for noun_case in case_list:
          for number in sing_pl:
            data.append(get_latin_form(word, case = noun_case, number = number))

        return pd.DataFrame(data=np.array(data).reshape(6, 2), index=case_list, columns={sing_pl[0]: 'Singular', sing_pl[1]: 'Plural'}).rename_axis('Case'), part_of_speech

      elif part_of_speech == "v" or part_of_speech == 't':
        person_list = ['first', 'second', 'third']
        for person in person_list:
          for number in sing_pl:
            data.append(get_latin_form(word, person = person, number = number, voice = 'active', mood='i', tense = "present"))

        return pd.DataFrame(data=np.array(data).reshape(3, 2), index=person_list, columns={sing_pl[0]: 'Singular', sing_pl[1]: 'Plural'}).rename_axis("Person"), part_of_speech
      elif part_of_speech == "a":
        data = {'word': []}
        case_list = ['nominative', 'genative', 'dative', 'accusative', 'ablative', 'vocative']
        gender_list = ['masculine', 'feminine', 'neuter']
        multi_index = pd.MultiIndex.from_product([case_list, sing_pl, gender_list], names=['case', 'number', 'gender'])
        for noun_case in case_list:
          for number in sing_pl:
            for gender in gender_list:
              data['word'].append(get_latin_form(word, case = noun_case, number = number, gender = gender))
        df = pd.DataFrame(data, index=multi_index)
        return df, part_of_speech
    word_df = latin_dict(word)
    print(type(word_df))
    if word_df[1] == None:
      return None

    if word_df[1] == "v" or word_df[1] == "t":
      return word_df[0], "verb"

    elif word_df[1] == "n":
      return word_df[0], "noun"

    elif word_df[1] == "a":
      return word_df[0], "adjective"

    else:
      return word_df, "none"


      # elif part_of_speech == "a":
      #   case_list = ['nominative', 'genative', 'dative', 'accusative', 'ablative', 'vocative']
      #   for case in case_list:
      #     for number in sing_pl:
      #       data.append(get_latin_form(word, case = case, number = number))


  df, part_of_speech = make_df(word)



#print(make_df(word))


# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy

      ##display(df)
      ##CREATE A FOLDER
  root = 'latin_variations_new'
  os.makedirs(f'{root}/{part_of_speech}/{word}/',exist_ok=True)
  # NAN ARE REPALCED BY SPACE CHARACTER
  df.to_csv(f'{root}/{part_of_speech}/{word}/{word}.csv')

  print(part_of_speech)
  if(show):display(df)

  ## READ
  #print('READ FROM FILE ------------------------------------')
  #display(pd.read_csv(f'{root}/{h3[i]}/{word}/{nav[i]}_{h4[i]}.csv'))
        #----------------------------------
#generate_variations('Ῥωμαῖος','greek',show=True)
#servus  and 'peregrinus', liber broke/are adjectives
#nouns = ['rex', 'pater', 'puella', 'vir', 'femina', 'puer',
#         'mater', 'regina', 'servus', 'civis', 'imperator',
#         'corpus', 'socer', 'agricola', 'peregrinus',
#         'miles', 'liber']
nouns = ['rex', 'pater', 'puella', 'vir', 'femina', 'puer',
         'mater', 'regina', 'civis', 'imperator',
         'corpus', 'socer', 'agricola',
         'miles']

# 'diligo' gives error if in array
verbs = ['diligo', 'defendo', 'servo', 'odi', 'neco', 'pugno',
         'sum', 'servamus']

adjectives = ["bonus",
"gratus", "fortis", "celer", "malus",
"rudis", "crudelis"]


for adjective in adjectives:
  generate_variations(adjectives, "latin", show = True)

for verb in verbs:
  generate_variations(verb,'latin',show=True)

for noun in nouns:
  generate_variations(noun,'latin',show=True)



#test words
#'Ῥωμαῖος' romans
#'μισώ' hate
#'εἰμί' is/are to be
#ελληνικά greeks
#odium hatred
#romanus
#odio hate
#απεχθάνομαι
#lacedaemon

# TO SHOW THE DATAFRAME PASS SHOW AS TRUE





ValueError: ('Lengths must match to compare', (17573,), (7,))

In [27]:


###### HELPER FUNCTIONS#######


def path_to_folder(path):
    # converts non folders into folders
    #aka just adds a slash to the end, if you dont give it a folder then it will probably produce an errror
    root = f"{path}"
    if root[-1] != "/":
        root+="/"
    return root


def get_each_file_in_folder(path):
    # this only takes in a folder path
    #RETURNS A FILE LIST
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            filelist.append(f)
    return filelist



def get_each_csv_in_folder(path):
    # takes in Noun, Adjective,Verb
    #Returns All csv's in a folder
    filelist =[]
    root =path_to_folder(path)
    for subdir, dirs, files in os.walk(root):
        for file in files:
            f = os.path.join(subdir, file)
            if f.endswith('.csv'):
                filelist.append(f)
    return filelist

def get_each_df_in_folder(path):
    #Takes in a folder
    #Gets every dataframe in a folder
    dflist =[]
    root =path_to_folder(path)
    for f in get_each_csv_in_folder(path):
        df = pd.read_csv(f).fillna('')

        dflist.append(df)
    return dflist
'''
def get_greek_df(path):
    # takes in Noun, Adjective,Verb
    #print("test")
    dflist =[]
    root =to_folder(path)
    for f in get_each_csv_in(path):
        df = pd.read_csv(f)
        greekdf=GreekCsv(f,df)
        dflist.append(greekdf)
    return dflist
'''
# get greek words from dictionary
def get_words_from_dict(path):
    wordlist = []
    for df in get_each_df_in_folder(path):
        for i in df.index:
             wordlist.append(df.iloc[i,1])
    return wordlist

#reads all csv and intializes all words into variations
def init_variations(path):
    # takes in a folder path
    # goes though every word in a dictionary and generates the variations of that word
    for word in get_words_from_dict(path):
        #display(df.iloc[i,1])
        generate_variations(word)

def match_words_in_folder(searchpath,wordpath):
#takes in a search path and a word path it will read a word from the word path and then find all
#files in each folder that contains that word
    filelist = []
    searchpath= path_to_folder(searchpath)
    for word in get_words_from_dict(wordpath):
        if(os.path.exists(searchpath + word)):
            filelist.append(get_each_file_in_folder(searchpath + word))
    flat_list = [item for sublist in filelist for item in sublist]
    return flat_list

def get_each_df_from_path_list(filelist, noun=False, verb=False):
    #takes in a list of csv paths and outputs a list of dataframes
    dflist=[]
    for file in filelist:
        df = pd.read_csv(file).fillna('')
        df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
        df.rename(columns=lambda x: x.lower(), inplace=True)
        dflist.append(df)

    dfreturn = []
    if(noun):
      for df in dflist:
        if(df.loc[df['case'] == "nominative", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['case'] == "nominative", "plural"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['case'] == "accusative", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if df.loc[df['case'] == "accusative", "plural"].values[0] == "error: nothing found with those parameters":
          continue
        if(df.loc[df['case'] == "dative", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['case'] == "dative", "plural"].values[0] == "error: nothing found with those parameters"):
          continue
        dfreturn.append(df)
    elif (verb):
      for df in dflist:
        if(df.loc[df['person'] == "first", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['person'] == "first", "plural"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['person'] == "second", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['person'] == "second", "plural"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['person'] == "third", "singular"].values[0] == "error: nothing found with those parameters"):
          continue
        if(df.loc[df['person'] == "third", "plural"].values[0] == "error: nothing found with those parameters"):
          continue
        dfreturn.append(df)
    return dfreturn

def display_list(list):
# takes in a list of dataframes and displays the data
    for i in list:
        display(i)

def display_all():
    init_variations("dictionary/")
    matchedwords= match_words_in_folder("variations/")
    display_list(get_each_df_from_path_list(matchedwords))




def get_indices(df,words):
    # returns a tuple of 2 lists that contain the row indexes and column inexes for each column and row that this word is present in
    # convert to take in a list of words
    columns=[]
    rows = []
    for columnIndex in range(df.shape[1]):
        if(isinstance(words, list)):
            for word in words:
                rowIndex = df.index[df.iloc[:, columnIndex].str.contains(word)].tolist()
                if(rowIndex):
                    columns.append(columnIndex)
                    rows.append(rowIndex)
        else:
            rowIndex = df.index[df.iloc[:, columnIndex].str.contains(words)].tolist()
            if(rowIndex):
                columns.append(columnIndex)
                rows.append(rowIndex)

            #remove duplicates

    rows = [item for sublist in rows for item in sublist]
    rows = [*set(rows)]
    #print("COLUMNINDEXES ",columns)
    #print("ROWINDEXES ",rows)
    return (rows,columns)


def get_items(dflist,row_words,column_words=''):
    # takes in a tuple of index arrays
    # empty strings are used to indicate not input standard input is either a string or array of strings
    dflist2=[]
    if (row_words=='' and column_words==''):
            print('please put a word to search in columns or rows')
            return []

    for df in dflist:#
        row_indexes = get_indices(df,row_words)[0]
        column_indexes = get_indices(df,column_words)[1]
        if (row_words=='' and column_words!=''):
            df2 =df.iloc[:,column_indexes]
        if (row_words!='' and  column_words==''):
            df2 =df.iloc[row_indexes]

        df2 =df.iloc[row_indexes,column_indexes]

        dflist2.append(df2)
    return dflist2

# a function that checks if a string contains a number
def contains_number(string):
    return any(char.isdigit() for char in string)


# not activley used will generate every possible combination kinda slow
def generate_all_accusative_scentences(verbs,nouns,sentiment):
    # takes in a verb dflist and noundf list aswell as asentiment
    sentiment = sentiment.lower()
    with open(f"latin_accusative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:
        for verbdf in verbs:
            for column_index in range(1,3):
                for row_index in range(verbdf.shape[0]):
                    verb =verbdf.iat[row_index,column_index]
                    if(verb=='' or contains_number(verb)):
                        continue
                    for noundf in nouns:
                        for column_index2 in range(1,noundf.shape[1]):
                            for row_index2 in range(noundf.shape[0]):
                                noun =noundf.iat[row_index2,column_index2]

                                if(noun== '' or contains_number(noun)):
                                    continue
                                for word in verb.split(" "):
                                    f.write(word+" O -100\n")
                                    #print(word+" O -999")
                                for i,word in enumerate(noun.split(" ")):
                                    if(sentiment =="positive" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Positive\n")
                                            #print(word +" B-ASP Positive")
                                        else:
                                            f.write(word +" I-ASP Positive\n")
                                            #print(word +" I-ASP Positive")
                                    elif(sentiment =="negative" ):
                                        if(i==0):
                                            f.write(word +" B-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                        else:
                                            f.write(word +" I-ASP Negative\n")
                                            #print(word +" B-ASP Negative")
                                f.write(". O -100\n\n")
                                #print(". O -999")

# WHATS MAINLY USED FOR NOW    -------------------------------------------------------------------------

def generate_random_accusative_dataset(verbs,nouns,sentiment,num):
    # takes in a verb dflist and noundf list aswell as asentiment
    # takes in number of examples to output
    #randomly selects a verb noun pair
    with open(f"latin_accusative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:

        random_verb_dfs = random.choices(verbs,k=num)
        random_noun_dfs = random.choices(nouns,k=num)

        accusative_noun_dfs = []
        nominative_noun_dfs = []

        for k in random_noun_dfs:
          if k.index[0] == 3:
            case_value = k.loc[3, "Case"]
          elif k.index[0] == 0:
            case_value = k.loc[0, "Case"]
          if case_value == "Nominative":
            nominative_noun_dfs.append(k)
          elif case_value == "Accusative":
            accusative_noun_dfs.append(k)
          print()
          print()
          print()
         # display_list(accusative_noun_dfs)
        i = 0
        num_tries=0
        while i <num:
            random_verb_row = random.randrange(0,random_verb_dfs[i].shape[0])
            random_verb_column = random.randrange(1,random_verb_dfs[i].shape[1])
            nominative_noun_column = random_verb_column
            nominative_noun_row = 0
            random_noun_row = random.randrange(0,accusative_noun_dfs[i].shape[0])
            random_noun_column = random.randrange(1,accusative_noun_dfs[i].shape[1])


            verb =strip_accents_and_lowercase(
            random_verb_dfs[i].iat[ random_verb_row,random_verb_column])

            #  nominative noun (matches plural/singular of verb), random verb, random accusative noun

            noun =strip_accents_and_lowercase(random_noun_dfs[i].iat[ random_noun_row,random_noun_column])
            #print(verb)
            if((noun== '' or contains_number(noun)or
            verb=='' or contains_number(verb))
            and num_tries<20 ):
                print("COULDNT FIND NUMBER OF TRIES:", num_tries)
                num_tries+=1
                continue
            num_tries = 0
            f.write("[ O -999\n")# add brackets
            for word in verb.split(" "):
                f.write(word+" O -999\n")
            for j,word in enumerate(noun.split(" ")):
                if(sentiment =="positive" ):
                    if(j==0):
                        f.write(word +" B-ASP Positive\n")
                        #print(word +" B-ASP Positive")
                    else:
                        f.write(word +" I-ASP Positive\n")
                        #print(word +" I-ASP Positive")
                elif(sentiment =="negative" ):
                    if(j==0):
                        f.write(word +" B-ASP Negative\n")
                        #print(word +" B-ASP Negative")
                    else:
                        f.write(word +" I-ASP Negative\n")
                        #print(word +" B-ASP Negative")
            f.write("] O -999\n\n")
            #randomly select item or randomly select row and column
            #check if its valid if its not randomly get another pai
            i+=1

#WHATS MAINLY USED FOR NOW
def generate_random_nominative_dataset(noun_dfs,verb_dfs,adjective_dfs,sentiment,num):
    # takes in a verb dflist and noundf list aswell as asentiment
    # takes in number of examples to output
    #randomly selects a verb noun pair
    #mathc linking plurality
    with open(f"latin_nominative_{sentiment}.txt.atepc","w+",encoding="utf-8") as f:
        random_noun_dfs = random.choices(noun_dfs,k=num)
        if(isinstance(verb_dfs, list)):
            random_verb_dfs = random.choices(verb_dfs,k=num)
        random_adjective_dfs = random.choices(adjective_dfs,k=num)
        gramatical_number=["1 sg","2 sg","3 sg","1 pl","2 pl","3 pl"]

        #
        i = 0
        num_tries=0
        while i <num:
            random_gramatical_number = random.choice(gramatical_number)


            random_noun_row = random.randrange(0,random_noun_dfs[i].shape[0])
            random_noun_column = random.randrange(1,random_noun_dfs[i].shape[1])
            #random_noun_column = get_indices(random_noun_dfs[i],random_gramatical_number)

            #random_verb_row = get_indices(random_verb_dfs[i],random_gramatical_number)
            if(isinstance(verb_dfs, list)):
                random_verb_row = random.randrange(0,random_verb_dfs[i].shape[0])
                random_verb_column = random.randrange(1,random_verb_dfs[i].shape[1])

            random_adjective_row = random.randrange(0,random_adjective_dfs[i].shape[0])
            random_adjective_column = random.randrange(1,random_adjective_dfs[i].shape[1])

            noun =strip_accents_and_lowercase(
            random_noun_dfs[i].iat[ random_noun_row,random_noun_column])

            if(isinstance(verb_dfs, list)):
                verb =random_verb_dfs[i].iat[ random_verb_row,random_verb_column]
            else:
                verb =verb_dfs

            adjective =strip_accents_and_lowercase(
            random_adjective_dfs[i].iat[ random_adjective_row,random_adjective_column])

            #print(verb)
            if((noun== '' or contains_number(noun)or
            verb=='' or contains_number(verb)or
            adjective=='' or contains_number(adjective))
            and num_tries<20 ):
                print("COULDNT FIND NUMBER OF TRIES:", num_tries)
                num_tries+=1
                continue
            num_tries = 0
            f.write("[ O -999\n")# add brackets so that stuff doesnt break when put into the
            for j,word in enumerate(noun.split(" ")):
                if(sentiment =="positive" ):
                    if(j==0):
                        f.write(word +" B-ASP Positive\n")
                        #print(word +" B-ASP Positive")
                    else:
                        f.write(word +" I-ASP Positive\n")
                        #print(word +" I-ASP Positive")
                elif(sentiment =="negative" ):
                    if(j==0):
                        f.write(word +" B-ASP Negative\n")
                        #print(word +" B-ASP Negative")
                    else:
                        f.write(word +" I-ASP Negative\n")
                        #print(word +" B-ASP Negative")
            for word in verb.split(" "):
                f.write(word+" O -999\n")
            for word in adjective.split(" "):
                f.write(word+" O -999\n")
            f.write("] O -999\n\n")
            #randomly select item or randomly select row and column
            #check if its valid if its not randomly get another pai
            i+=1
# FUNCTION GETS A RANDOM SAMPLE OUTPUTS AN ARRAY OF STUFF
def random_sample(path,num):
#READS A RANDOM SAMPLE FROM A FILE AND APPENDS IT TO AN ARRAY
    sample = []
    with open(path,"r",encoding="utf-8") as f:

        data = f.read().split('\n\n')
        for i in range (num):
            sample.append(random.choice(data))

        # read untill double new line
        # append string to array
    return sample

def sample_to_dataset(sample,name):
# ideally every entry inside of the array should be a complete dataset entry ready to be outputted directly into a file one after another
    with open(name,"+w",encoding="utf-8") as f:
        for s in(sample):
            f.write(f"{s}\n\n")

def shuffle_datasets(paths,type='test',num=2000):
#shuffle all datasets in the paths given to it and then outputs it into a new file
    combined = []
    for p in paths:
        pos =random_sample(f"{p[0]}",int(num/(2*len(paths))))
        neg =random_sample(f"{p[1]}",int(num/(2*len(paths))))
        combined += pos+neg
    random.shuffle(combined)
    sample_to_dataset(combined,f"latin.{type}.txt.atepc")




#nomitiv
#negation
#conjunction



In [ ]:
# test function to create accusative sentences
# instead of splitting nouns into seperate dataframes, it simply chooses the version of the noun
# based on the version of the verb randomly chosen
#subject first then accusative noun then verb

def generate_random_accusative_dataset_v2(verbs,nouns,num,sentiment=''):
  sentences = []
  for i in range(num):
    # randomly chooses form of verb and what words are plural/singular
    verb_plurality = random.choice(["singular", "plural"])
    verb_person = random.choice(["first","third"])
    accusative_plurality = random.choice(["singular", "plural"])


    # chooses dataframes making sure that nouns are not the same
    accusative_index = random.randint(0, len(nouns)-1)
    nominative_index = random.randint(0, len(nouns)-1)
    while(accusative_index == nominative_index):
      accusative_index = random.randint(0, len(nouns)-1)
      nominative_index = random.randint(0, len(nouns)-1)
    accusative_noun_df = nouns[accusative_index]
    nominative_noun_df = nouns[nominative_index]
    verb_df = random.choice(verbs)

    # gets correct declensions of dataframes
    verb = verb_df.loc[verb_df['person'] == verb_person, verb_plurality].values[0]
    nominative_noun = nominative_noun_df.loc[nominative_noun_df['case'] == "nominative", verb_plurality].values[0]
    accusative_noun = accusative_noun_df.loc[accusative_noun_df['case'] == "accusative", accusative_plurality].values[0]

    # makes the sentence gramatically correct
    if(verb_person == "first"):
      sentence = accusative_noun + " " + verb
    if(verb_person == "third"):
      sentence = nominative_noun + " " + accusative_noun + " " + verb
    sentences.append(sentence)

  return sentences



def generate_random_coordinating_conjunction_accusative_dataset(verbs,nouns,num,sentiment=''):
  sentences = generate_random_accusative_dataset_v2(verbs,nouns,num*2)
  conjunctive_sentences = []
  for i in range(0, len(sentences), 2):
    sentence = sentences[i] + " " + random.choice(["et", "sed", "ergo"]) + " " + sentences[i+1]
    conjunctive_sentences.append(sentence)
  return conjunctive_sentences

def generate_random_correlative_conjunction_accusative_dataset(verbs,nouns,num,sentiment=''):
  sentences = []
  for i in range(num):
    # randomly chooses form of verb and what words are plural/singular
    verb_plurality = random.choice(["singular", "plural"])
    verb_person = random.choice(["first","third"])

    # chooses dataframes making sure that nouns are not the same
    accusative_index_1 = random.randint(0, len(nouns)-1)
    accusative_index_2 = random.randint(0, len(nouns)-1)
    nominative_index = random.randint(0, len(nouns)-1)
    while(accusative_index_1 == nominative_index and accusative_index_1 == accusative_index_2):
      accusative_index_1 = random.randint(0, len(nouns)-1)
      accusative_index_2 = random.randint(0, len(nouns)-1)
      nominative_index = random.randint(0, len(nouns)-1)


    accusative_noun_df_1 = nouns[accusative_index_1]
    accusative_noun_df_2 = nouns[accusative_index_2]
    nominative_noun_df = nouns[nominative_index]
    verb_df = random.choice(verbs)

    # gets correct declensions of dataframes
    print(verb_df)
    verb = verb_df.loc[verb_df['person'] == verb_person, verb_plurality].values[0]
    nominative_noun = nominative_noun_df.loc[nominative_noun_df['case'] == "nominative", verb_plurality].values[0]
    accusative_noun_1 = accusative_noun_df_1.loc[accusative_noun_df_1['case'] == "accusative", random.choice(["singular", "plural"])].values[0]
    accusative_noun_2 = accusative_noun_df_2.loc[accusative_noun_df_2['case'] == "accusative", random.choice(["singular", "plural"])].values[0]


    # makes the sentence gramatically correct
    if(verb_person == "first"):
      sentence = accusative_noun_1 + " et " + accusative_noun_2 + " " + verb
    if(verb_person == "third"):
      sentence = nominative_noun + " " + accusative_noun_1 + " et " + accusative_noun_2 + " " + verb
    sentences.append(sentence)
  return sentences



#
def generate_random_predicate_nominative_sentence(verbs,nouns,num,sentiment=''):
  sentences = []
  for i in range(num):
    # randomly chooses form of verb and what words are plural/singular
    plurality = random.choice(["singular", "plural"])
    person = random.choice(["first","third"])


    # chooses dataframes making sure that nouns are not the same
    nominative_index_1 = random.randint(0, len(nouns)-1)
    nominative_index_2 = random.randint(0, len(nouns)-1)
    while(nominative_index_1 == nominative_index_2):
      nominative_index_1 = random.randint(0, len(nouns)-1)
      nominative_index_2 = random.randint(0, len(nouns)-1)
    nominative_noun_df_1 = nouns[nominative_index_1]
    nominative_noun_df_2 = nouns[nominative_index_2]

    # gets correct declensions of dataframes
    print(nominative_noun_df_1)
    nominative_noun_1 = nominative_noun_df_1.loc[nominative_noun_df_1['case'] == "nominative", plurality].values[0]
    print(nominative_noun_df_2)
    nominative_noun_2 = nominative_noun_df_2.loc[nominative_noun_df_2['case'] == "nominative", plurality].values[0]

    # makes the sentence gramatically correct
    if(person == "first" and plurality == "singular"):
      sentence = nominative_noun_1 + " sum " + nominative_noun_2
    if(person == "first" and plurality == "plural"):
      sentence = nominative_noun_1 + " sumus " + nominative_noun_2
    if(person == "third" and plurality == "singular"):
      sentence = nominative_noun_1 + " est " + nominative_noun_2
    if(person == "third" and plurality == "plural"):
      sentence = nominative_noun_1 + " stunt " + nominative_noun_2
    sentences.append(sentence)
  return sentences

def generate_random_dative_accusative_sentence(verbs,nouns,num,sentiment=''):
  sentences = []
  for i in range(num):
    # randomly chooses form of verb and what words are plural/singular
    plurality = random.choice(["singular", "plural"])
    person = random.choice(["first","third"])


    # chooses dataframes making sure that nouns are not the same
    dative_index = random.randint(0, len(nouns)-1)
    accusative_index = random.randint(0, len(nouns)-1)
    while(dative_index == accusative_index):
      dative_index = random.randint(0, len(nouns)-1)
      accusative_index = random.randint(0, len(nouns)-1)
    dative_noun_df = nouns[dative_index]
    accusative_noun_df = nouns[accusative_index]

    # gets correct declensions of dataframes
    dative_noun = dative_noun_df.loc[dative_noun_df_1['case'] == "dative", random.choice(["singular", "plural"])].values[0]
    accusative_noun = accusative_noun_df.loc[nominative_noun_df_2['case'] == "accusative", random.choice(["singular", "plural"])].values[0]
    verb = verbs[random.randint(0, len(verbs)-1)].loc[verb_df['person'] == person, plurality].values[0]

    sentence = accusative_noun + " " + dative_noun + " " + verb
    sentences.append(sentence)

  return sentences





matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")
matchedverbs_pos= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs")
#----------ALL DATA FRAMES IN LOWERCASE----------
nouns=get_each_df_from_path_list(matchednouns, noun=True, verb=False)
verbs = get_each_df_from_path_list(matchedverbs_pos, noun=False, verb=True)


for sentence in (generate_random_accusative_dataset_v2(verbs,nouns,20)):
  print(sentence)
print("----------random accusative----------")
for sentence in (generate_random_accusative_dataset_v2(verbs,nouns,20)):
  print(sentence)
print("----------coordinating conjunctions----------")
for sentence in (generate_random_coordinating_conjunction_accusative_dataset(verbs,nouns,20)):
  print(sentence)
print("----------correlative conjunctions----------")
for sentence in (generate_random_correlative_conjunction_accusative_dataset(verbs,nouns,20)):
  print(sentence)

# print("----------Predicate Nominative----------")
# for sentene in (generate_random_predicate_nominative_sentence(verbs,nouns,20)):
#   print(sentence)
#Nominative nouns: nominative noun + to be verb + adjective matches noun


servi matres seruant
miles reginas seruat
matrem diligo
regem seruo
agricolae puerum uident
puerum defendo
liber feminam capit
ciuem sumus
ciuem sum
servi regen capiunt
puer liberum seruat
matres servamus
miles inperatorem uidet
milites inperatores defendunt
peregrinos audimus
agricola puellam audit
servos defendo
liberos pugnamus
ciuem odi
agricolam videone
----------random accusative----------
puellas servamus
servus reginas capit
liber pueros pugnat
patrem odi
inperatores odi
agricolas sum
ciuem pugno
inperator servum est
servi peregrinos diligunt
agricolae ciuis seruant
matrem audimus
puella patres odi
ciuem sumus
regina feminam odi
ciuem pugnamus
pater regen pugnat
regem odi
patres diligo
peregrinos capio
servus regen seruat
----------coordinating conjunctions----------
liberum uidemus ergo puellas servamus
peregrini servos uident et miles feminam seruat
femina peregrinum uidet et puer patres odi
servos pugnamus ergo inperator regen uidet
pater matres odi sed reginam odi
regina fe

In [ ]:
#get_each_file_in("dictionary/nouns")
#get_each_csv_in("dictionary/nouns")
#init_variations_in("dictionary/nouns")
#matchedwords= match_words_in("variations/noun","dictionary/nouns")

# THIS FUNCTION PIECES TOGETHER A RANDOM SAMPLE OF DECLINED WORDS TO CREATE A ACCUSATIVE SCENTENCE
def test_create_accusative(init=False,randomize=True,debug=False):
    # INIIT VARAIATIONS CREATES THE GREEK VARIATIONS FOLDER
    if (init==True or not os.path.exists("latin_variations_new/")):
        init_variations("latin_dictionary/nouns")
        init_variations("latin_dictionary/verbs/actionverbs/negative")
        init_variations("latin_dictionary/verbs/actionverbs/positive")
        #init_variations_in("greek_dictionary/adjectives/positive")
        #init_variations_in("greek_dictionary/adjectives/negative")


    #'''
    # searches the words in the greek varations folder for nouns
    matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")

    #display_list(get_accustive_nouns(path_to_df(matchednouns)))


    matchedverbs_pos= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/actionverbs/positive")
    print(matchedverbs_pos)
    matchedverbs_neg= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/actionverbs/negative")
    print(matchedverbs_neg)
    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))
    nouns=get_items(get_each_df_from_path_list(matchednouns),("Accusative")) + get_items(get_each_df_from_path_list(matchednouns),("Nominative"))

    print(f"Nouns {nouns}")
    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    verbs_pos = get_items(get_each_df_from_path_list(matchedverbs_pos),"Third Person")#, "Singular")
   # print(f"get_each_df_from_path_list {get_each_df_from_path_list(matchedverbs_pos)}")
    print(f"Verbs_Pos {verbs_pos}")
    verbs_neg=get_items(get_each_df_from_path_list(matchedverbs_neg),"Third Person")#, "Singular")
   # print(f"get_each_df_from_path_list {get_each_df_from_path_list(matchedverbs_neg)}")
    print(f"Verbs_neg {verbs_neg}")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))

    #display(nouns)
    if debug:display_list(nouns)
    if debug:display_list(verbs_pos)
    if(randomize == True):

      print('test')
      generate_random_accusative_dataset(verbs_pos,nouns,"positive",1000)
      generate_random_accusative_dataset(verbs_neg,nouns,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")
      '''
      neg =random_sample("accusative_negative.txt.atepc",1000)
      pos=random_sample("accusative_positive.txt.atepc",1000)
      combined = neg+pos
      random.shuffle(combined)
      sample_to_dataset(combined,"greek.test.txt.atepc")


      generate_random_accusative_dataset(verbs_pos,nouns,"positive",1000)
      generate_random_accusative_dataset(verbs_neg,nouns,"negative",1000)

      neg2 =random_sample("accusative_negative.txt.atepc",1000)
      pos2=random_sample("accusative_positive.txt.atepc",1000)
      combined2 = neg2+pos2
      random.shuffle(combined2)
      sample_to_dataset(combined2,"greek.train.txt.atepc")
      '''
    else:
      generate_all_accusative_scentences(verbs_pos,nouns,"positive")
      generate_all_accusative_scentences(verbs_neg,nouns,"negative")


#---------------------------
# THIS FUNCTION PIECES TOGETHER A RANDOM SAMPLE OF DECLINED WORDS TO CREATE A NOMINATIVCE SCENTENCE
def test_create_nominative(init=False,randomize=True,debug=False):
    if (init==True or not os.path.exists("latin_variations_new/")):
        init_variations("latin_dictionary/nouns")
        init_variations("latin_dictionary/verbs/linkingverbs/")
        init_variations("latin_dictionary/adjectives/positive")
        init_variations("latin_dictionary/adjectives/negative")


    #'''

    matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")

    #display_list(get_accustive_nouns(path_to_df(matchednouns)))


    matchedverbs= match_words_in_folder("latin_variations_new/verb","latin_dictionary/verbs/linkingverbs/")
    matched_adj_pos= match_words_in_folder("latin_variations_new/adjective","latin_dictionary/adjectives/positive")
    matched_adj_neg= match_words_in_folder("latin_variations_new/adjective","latin_dictionary/adjectives/negative")


    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))

    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    nouns=get_items(get_each_df_from_path_list(matchednouns),'nominative')
    #verbs=get_items(path_to_df(matchedverbs),["sg","pl"],"present")
    #display_list(path_to_df(matchedverbs))
    #verbs=get_items(path_to_df(matchedverbs),"infinitive","active")
    verbs= strip_accents_and_lowercase("sum")

    adjectives_pos=get_items(get_each_df_from_path_list(matched_adj_pos),"nominative")
    adjectives_neg=get_items(get_each_df_from_path_list(matched_adj_neg),"nominative")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))

    #display(nouns)
    #if debug:display_list(nouns)
    #if debug:display_list(verbs)
    #if debug:display_list(adjectives_pos)
    if(randomize == True):
        generate_random_nominative_dataset(nouns,verbs,adjectives_pos,"positive",1000)
        generate_random_nominative_dataset(nouns,verbs,adjectives_neg,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")
        '''
        neg =random_sample("nominative_negative.txt.atepc",1000)
        pos=random_sample("nominative_positive.txt.atepc",1000)
        combined = neg+pos
        random.shuffle(combined)
        sample_to_dataset(combined,"greek.test.txt.atepc")
        '''
        '''
        generate_random_nominative_dataset(nouns,verbs,adjectives_pos,"positive",1000)
        generate_random_nominative_dataset(nouns,verbs,adjectives_neg,"negative",1000)
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")

        neg =random_sample("nominative_negative.txt.atepc",1000)
        pos=random_sample("nominative_positive.txt.atepc",1000)
        combined = neg+pos
        random.shuffle(combined)
        sample_to_dataset(combined,"greek.train.txt.atepc")
        '''

    else:
        pass
        #generate_all_accusative_scentences(verbs_pos,nouns,"positive")
        #generate_all_accusative_scentences(verbs_neg,nouns,"negative")


In [28]:
# TEST GENERATE IS A FUNCTION THAT CREATE
def testGenerate(filename ='test', initialize = False):
    # init is a parmeter that generates all of the declension csv's
    #right now its a little innefficiant if the csv's are already initialized then you do not need to initialize so you can set to false and it will generate almost immediatley
     test_create_accusative(
     init=initialize,
     randomize=True,debug=True)

    # test_create_nominative(
    # init = initialize,
   #  randomize=True,debug=True)

     dataset_paths=[
          ["latin_accusative_positive.txt.atepc","latin_accusative_negative.txt.atepc"],
          #["latin_nominative_positive.txt.atepc","latin_nominative_negative.txt.atepc"]
          ]
     shuffle_datasets(dataset_paths,type=filename,num= 2000)

testGenerate(filename ='test',initialize = True) # set initialize to false if this is not your first time running for it to generate fast
testGenerate(filename ='train',initialize = True)

#this generates atepc files just open in a text editor download a text editor plugin for google drive to make it easier


n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
a
<class 'tuple'>
adjective
n
<class 'tuple'>
noun
a
<class 'tuple'>
adjective
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
a
<class 'tuple'>
adjective
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
n
<class 'tuple'>
noun
v
<class 'tuple'>
verb
v
<class 'tuple'>
verb
v
<class 'tuple'>
verb
t
<class 'tuple'>
verb
v
<class 'tuple'>
verb
v
<class 'tuple'>
verb
['latin_variations_new/verb/diligo/diligo.csv', 'latin_variations_new/verb/defendo/defendo.csv', 'latin_variations_new/verb/servo/servo.csv']
['latin_variations_new/verb/odi/odi.csv', 'latin_variations_new/verb/neco/neco.csv', 'latin_variations_new/verb/pugno/pugno.csv']
Nouns []
Verbs_Pos []
Verbs_neg []
test


IndexError: list index out of range

In [ ]:
pd.read_csv("latin_variations_new/verb/diligo/_conjugation.csv")

In [ ]:
pd.read_csv("latin_variations_new/noun/aegyptus/_declension.csv")

In [ ]:
matchednouns= match_words_in_folder("latin_variations_new/noun","latin_dictionary/nouns")
nouns=get_items(get_each_df_from_path_list(matchednouns), "accusative", "")
count = 0
for item in nouns:
  count += 1
print('\n')
for item in nouns:
  try:
    print(item["Singular"])
  except:
      print(item["1"])
      print(item["2"])
      print(item["3"])

  #print(item["Singular"])

#print(nouns[0].iloc[0])
#print(nouns[0].accusative['Singular'])
for match in lookup.match_word('casus'): # Match possible words
    print(lookup.get_dictionary_string(match))  # Start with the definition of the word
    for i in lookup.get_word_inflections(match,less=False):
        print(i) # Then print each inflection's meaning
        print()



In [ ]:
!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words
def make_df(word):
  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])


  # runs if the word has no gendered attributes
  if gender == {"-"}:
    singular_dict = {}
    plural_dict = {}
    for i in range(0,len(word)):
      # check if noun
      if word[i][1][0] == "-":
        # check if singular
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])

    dict = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict["Singular"].append(singular_dict["Nominative"])
    dict["Singular"].append(singular_dict["Genative"])
    dict["Singular"].append(singular_dict["Dative"])
    dict["Singular"].append(singular_dict["Accusative"])
    dict["Singular"].append(singular_dict["Ablative"])
    dict["Singular"].append(singular_dict["Vocative"])

    dict["Plural"].append(plural_dict["Nominative"])
    dict["Plural"].append(plural_dict["Genative"])
    dict["Plural"].append(plural_dict["Dative"])
    dict["Plural"].append(plural_dict["Accusative"])
    dict["Plural"].append(plural_dict["Ablative"])
    dict["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict)
    return df
  else:
    print("not valid at the moment")
    return -1


for item in decline("puella"):
  print(item)

word = decline("puella")
print(word)
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy
print(make_df(word))




In [ ]:
#!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words

def fill_dicts(word):
    singular_dict = {}
    plural_dict = {}
    for i in range(0, len(word)):

        # check if singular
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])
    return singular_dict, plural_dict




def make_df(word):
  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])
  print(gender)
  # runs if the word has no gendered attributes
  if gender == {"-"}:
    singular_dict, plural_dict = fill_dicts(word)
    print(singular_dict, plural_dict)
    print()
    dict = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict["Singular"].append(singular_dict["Nominative"])
    dict["Singular"].append(singular_dict["Genative"])
    dict["Singular"].append(singular_dict["Dative"])
    dict["Singular"].append(singular_dict["Accusative"])
    dict["Singular"].append(singular_dict["Ablative"])
    dict["Singular"].append(singular_dict["Vocative"])

    dict["Plural"].append(plural_dict["Nominative"])
    dict["Plural"].append(plural_dict["Genative"])
    dict["Plural"].append(plural_dict["Dative"])
    dict["Plural"].append(plural_dict["Accusative"])
    dict["Plural"].append(plural_dict["Ablative"])
    dict["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict)
    return df

  elif gender == {'f','n','m'}:
    # fills each list with the respective words
    masculine, feminine, neutral = [], [], []
    for item in word:
      if item[1][6] == 'm':
        masculine.append(item)
      elif item[1][6] == 'f':
        feminine.append(item)
      else:
        neutral.append(item)

  else:
    print("not valid at the moment")
    return -1


# for item in decline("Romanus"):
#   print(item)

word = decline("Romanus")
print(word)
print(type(word))
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy
print(make_df(word))
print('hi')



In [ ]:
!pip install cltk
import pandas as pd
from cltk.morphology.lat import CollatinusDecliner
from cltk.morphology.akk import get_bound_form
from cltk.morphology.akk import decline_noun

from cltk.data.fetch import FetchCorpus
corpus_importer = FetchCorpus("lat")
print(corpus_importer.list_corpora)
corpus_importer.import_corpus("lat_models_cltk")

# takes in a word as a string and uses the collatinus decliner to return
# the different forms of the word. returns words which is a list that contains
# tuples formated like so: [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
def decline(word):
  decliner = CollatinusDecliner()
  words = decliner.decline(word)
  return words

# takes in a declined word object (list of words in the format returned by the decliner function) and
# formats it so that each word is with the correct type (nominative, genative, etc.)
# currently only works with nouns that have one gender. needs to be expanded so it can make
# dictionaries for multiple genders and word types.
def fill_dicts(word, neutral = False, masc = False, fem = False):
    singular_dict = {}
    plural_dict = {}
    for i in range(0, len(word)):

      #check if verb
      if word[i][1][0] == "v":

        #check if singular verb
        if word[i][1][1:] == "1spia---":
          singular_dict["First Person"] = (word[i][0])
        if word[i][1][1:] == "2spia---":
          singular_dict["Second Person"] = (word[i][0])
        if word[i][1][1:] == "3spia---":
          singular_dict["Third Person"] = (word[i][0])
        #check if plural verb
        if word[i][1][1:] == "1ppia---":
          plural_dict["First Person"] = (word[i][0])
        if word[i][1][1:] == "2ppia---":
          plural_dict["Second Person"] = (word[i][0])
        if word[i][1][1:] == "3ppia---":
          plural_dict["Third Person"] = (word[i][0])

      # check if singular
      elif word[i][1][0] == "-":
        if word[i][1][2] == "s":
          if word[i][1][7] == "n":
            singular_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            singular_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            singular_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            singular_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            singular_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            singular_dict["Vocative"] = (word[i][0])
        # check if plural
        elif word[i][1][2] == "p":
          if word[i][1][7] == "n":
            plural_dict["Nominative"] = (word[i][0])
          elif word[i][1][7] == "g":
            plural_dict["Genative"] = (word[i][0])
          elif word[i][1][7] == "d":
            plural_dict["Dative"] = (word[i][0])
          elif word[i][1][7] == "a":
            plural_dict["Accusative"] = (word[i][0])
          elif word[i][1][7] == "b":
            plural_dict["Ablative"] = (word[i][0])
          elif word[i][1][7] == "v":
            plural_dict["Vocative"] = (word[i][0])
    return singular_dict, plural_dict




def make_df(word, noun = False):

  gender = set()
  for i in range(0,len(word)):
    gender.add(word[i][1][6])
  # runs if the word has no gendered attributes
  #if '-' in gender or 'n' in gender:
  singular_dict, plural_dict = fill_dicts(word, True)
  print(singular_dict, plural_dict)
  print()

  if word[1][1][0] == "v":
      dict_verb = {"Person": ["First Person", "Second Person", "Third Person"], "Singular":[], "Plural":[]}
      dict_verb["Singular"].append(singular_dict["First Person"])
      dict_verb["Singular"].append(singular_dict["Second Person"])
      dict_verb["Singular"].append(singular_dict["Third Person"])
      dict_verb["Plural"].append(singular_dict["First Person"])
      dict_verb["Plural"].append(singular_dict["Second Person"])
      dict_verb["Plural"].append(singular_dict["Third Person"])
      df = pd.DataFrame(dict_verb)
      df = df.set_index("Person")
      return df

  if word[1][1][0] == "-":
    dict_overall = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}
    dict_overall["Singular"].append(singular_dict["Nominative"])
    dict_overall["Singular"].append(singular_dict["Genative"])
    dict_overall["Singular"].append(singular_dict["Dative"])
    dict_overall["Singular"].append(singular_dict["Accusative"])
    dict_overall["Singular"].append(singular_dict["Ablative"])
    dict_overall["Singular"].append(singular_dict["Vocative"])

    dict_overall["Plural"].append(plural_dict["Nominative"])
    dict_overall["Plural"].append(plural_dict["Genative"])
    dict_overall["Plural"].append(plural_dict["Dative"])
    dict_overall["Plural"].append(plural_dict["Accusative"])
    dict_overall["Plural"].append(plural_dict["Ablative"])
    dict_overall["Plural"].append(plural_dict["Vocative"])
    df = pd.DataFrame(dict_overall)
    return df
  if 'm' in gender:
        dict_masc = {"Case":["Nominative", "Genative", "Dative", "Accusative", "Ablative", "Vocative"], "Singular":[], "Plural":[]}

  else:
    print("not valid at the moment")
    print(gender)
    return -1


# for item in decline("ambulo"):
#   print(item)

#print(make_df(word))
word = decline("rex")
print(make_df(word))
df = make_df(word)
# formated by [word, (part of speech, person, number, tense, mood, voice, gender, case, degree)]
# https://github.com/bab2min/lamonpy

# print('hi')



In [ ]:
newdfdict = {
    "Case": ["Nominative"],
    "Singular": ["a"],
    "Plural": ["b"]
}

newdf = pd.DataFrame(newdfdict)

dfdf = newdf.loc[0,"Case"]
dfdf

In [ ]:
import pandas as pd


cases = ['nominative', 'genitive', 'dative', 'accusative', 'ablative', 'vocative']
numbers = ['singular', 'plural']
genders = ['masculine', 'feminine', 'neuter']


multi_index = pd.MultiIndex.from_product([cases, numbers, genders], names=['Case', 'Number', 'Gender'])

data = {'word': ["word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word",
                 "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word", "word"
                   ]}
data['word'].append("hi")

df_3d = pd.DataFrame(data, index=multi_index)

df_3d
